In [34]:
import os
import json
from pathlib import Path

In [35]:
DATA_RAW_DIR = 'data/raw'
DATA_RAW_FILE = 'tiendapet.jl'

project_dir = Path().cwd().parent
data_raw_path = project_dir.joinpath(DATA_RAW_PATH)
data_raw_file_path = data_raw_path.joinpath(DATA_RAW_FILE)

In [46]:
j_content = []
with open(data_raw_file_path) as f:
    for line in f:
        j_content.append(json.loads(line))

## Nutritional Facts table built
Parsing the field in each json line to accomodate a table that has (product_name - nutritional facts). Columns are fixed for now
* nf_df = nutritional facts data frame

In [136]:
import pandas as pd

In [137]:
nf_df = pd.DataFrame(columns=["product_name", "protein", "fat", "fiber", "humidity", "food_ashes", "calcium", "phosphorus"])

In [302]:
def filter_nutritional_fact_lines(data: list[dict]) -> list:
    """"""
    search = re.compile(r'[a-z0-9.]').search
    filtered_lines = []
    for line in data:
        if search(''.join(line.get('nutritional_facts', ''))):
            filtered_lines.append(line)
    return filtered_lines


In [311]:
def filter_protein_lines(nutritional_facts_list: list) -> str:
    """"""
    search = re.compile(r'prote\wnas?\s(?:\w+\s).*?(?:\()?((?:\d+)(?:.)?(?:\d+.?%))(?:\))?|prote\wnas?:?\s(?:\()?(\d+.?%)(?:\))?|prote\wnas?\s(?:\w+\s?:?).*?((?:\d+)(?:.)?(?:\d+.?%))|prote\wna?s?:?\s(\d+%)', re.MULTILINE | re.IGNORECASE).search
    p = None
    for _ in nutritional_facts_list:
        result = search(_)
        if result:
            p = ''.join([_ for _ in result.groups() if _])
            break
    search = re.compile(r'\bprote\wna?s?\b(?:\s)?(?:\w+)?', re.MULTILINE | re.IGNORECASE).search
    if not p:
        for idx, _ in enumerate(nutritional_facts_list):
            result = search(_)
            if result:
                try:
                    p = nutritional_facts_list[idx + 3]
                except IndexError:
                    p = None
                break
    return p

In [316]:
def protein_value_helper(protein_list:list) -> None:
    """manual helper for protein list """
    l = protein_list
    l[265] = '23%'
    return l
    

protein_list = []
for idx, line in enumerate(nf_lines):
    protein = filter_protein_lines(line["nutritional_facts"])
    protein_list.append(protein)

protein_list = protein_value_helper(protein_list)

In [317]:
for idx, item in enumerate(protein_list):
    if not item:
        print(idx, nf_lines[idx]["nutritional_facts"])

3 ['\n\t', 'Aditivos nutricionales:', '\n\t', '\n\tVitamina A 18.000 UI/kg. Vitamina D3 1.800 UI/kg. Vitamina E 200 mg/kg. Vitamina C 75 mg/kg. Hierro (sulfato de hierro (II), monohidratado) 75 mg/kg. Yodo (yoduro potásico) 3,50 mg/kg. Cobre (sulfato de cobre (II) pentahidratado) 10 mg/kg. Manganeso (sulfato manganoso monohidratado) 7,50 mg/kg. Zinc (óxido de zinc) 120 mg/kg. Selenio (selenito de sodio) 0,12 mg/kg.', '\n\t', '\n\t', 'Aditivos tecnológicos:', '\n\t', '\n\tAntioxidantes: Extractos de tocoferoles de aceites vegetales 100 mg/kg.', '\n\t', '\n\t', 'Guía de alimentación:', '\n', '\n\t', '\n\t\t', '\n\t\t\t', '\n\t\t\t\t', '\xa0 \xa0Peso\xa0 \xa0', '\n\t\t\t', '\n\t\t\t\t', 'Poca actividad', '\n\t\t\t', '\n\t\t\t\t', 'Actividad media', '\n\t\t\t', '\n\t\t\t\t', 'Alta actividad', '\n\t\t', '\n\t\t', '\n\t\t\t', '\n\t\t\t\t1kg', '\n\t\t\t', '\n\t\t\t\t24gr', '\n\t\t\t', '\n\t\t\t\t28gr', '\n\t\t\t', '\n\t\t\t\t35gr', '\n\t\t', '\n\t\t', '\n\t\t\t', '\n\t\t\t\t5kg', '\n\t\t\t', 

In [320]:
for idx, _ in enumerate(protein_list):
    print(idx, _)
    
27, 50, 57, 168, 544

0 29%
1 20 %
2 30 %
3 None
4 25%
5 25%
6 25.5 %
7 26,0%
8 27,0%
9 None
10 25%
11 34.9%
12 26%
13 29.0%
14 25,00%
15 24.5
16 None
17 29,0%
18 23,00%
19 25.0
20 24.5
21 28,0%
22 None
23 25,0%
24 42 %
25 00%
26 28,0%
27 
	Fibra   03.00%
28 None
29 26,0%
30 38 %
31 25.0
32 27 %
33 None
34 25,3
35 32.0%
36 25.0%
37 None
38 22,0%
39 28.5
40 24,0%
41 29,0%
42 28.0%
43 28.0%
44 32%
45 None
46 26%
47 26,0
48 28%
49 26.0%
50 
										
51 29%
52 12,0 %
53 
		
54 40%
55 28,6
56 28.0%
57 
		
58 
		
59 40%
60 21%
61 18.7
62 26%
63 30%
64 None
65 
		
66 38%
67 38%
68 38%
69 26%
70 
		
71 25%
72 30%
73 26,0%
74 None
75 None
76 26%
77 27%
78 25%
79 25%
80 29%
81 24%
82 12 %
83 28%
84 26%
85 
		
86 25,8
87 18.6
88 21.2
89 23.0%
90 37%
91 34,6
92 18.1
93 15,5
94 
		
95 20,7
96 20,7
97 17,7
98 24,9
99 29,8
100 29,8
101 29,5
102 
		
103 28%
104 26%
105 20,9
106 24.5
107 16%
108 19,4
109 17,2
110 18,3
111 
		
112 29%
113 28.0
114 28.1
115 23,0
116 18,0%
117 28%
118 16%
119 38.0%
120 38.0%
121 38.0%
122 4.